Import all the required libraries and download the wordnet and stopwords.

In [282]:
from sklearn.ensemble import RandomForestClassifier
import re
from nltk.corpus import stopwords as stop
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Read the reviews from the files.

In [283]:
trainNeg = open("imdb_train_neg.txt", "r").readlines()
trainPos = open("imdb_train_pos.txt", "r").readlines()

testPos = open("imdb_test_pos.txt", "r").readlines()
testNeg = open("imdb_test_neg.txt", "r").readlines()

devPos = open("imdb_dev_pos.txt", "r").readlines()
devNeg = open("imdb_dev_neg.txt", "r").readlines()

print(len(trainNeg))
print(len(testNeg))
testPos[:1]

7517
2501


['This is the greatest movie if you want inspiration on following your heart and never giving up on your dream. Elizabeth Taylor is Velvet and in her prime (of her childhood, at least), Mickey Rooney is a cynical friend who eventually becomes her trainer and they go off to the Grand National steeplechase with her beloved horse "the Pi"--short for "Pirate"--only to have Velvet become the jockey and have a chance at victory. To those of you who have not seen it yet, I won\'t give away the ending but you should see it and once you do you\'ll love it. Notice a very young Angela Lansbury as Velvet\'s eldest sister.\n']

Remove all the special characters from the dataset.

In [284]:
def cleanSpecialCharacters(dataset):
    specialchars = re.compile("[.;:!\'?,\"()\[\]]")
    html = re.compile("(<.*?>)|(\-)|(\/)")

    dataset = [specialchars.sub("", line) for line in dataset]
    dataset = [html.sub(" ", line) for line in dataset]
    
    return dataset

trainNeg = cleanSpecialCharacters(trainNeg)
trainPos = cleanSpecialCharacters(trainPos)

testNeg = cleanSpecialCharacters(testNeg)
testPos = cleanSpecialCharacters(testPos)

devPos = cleanSpecialCharacters(devPos)
devNeg = cleanSpecialCharacters(devNeg)

trainPos[:1]

['For fans of Chris Farley this is probably his best film David Spade plays the perfect cynical sarcastic yin to Farleys Baby Huey yang Farley achieves strokes of comic genius in his monologues like the Lets say youre driving along the road with your family bit the Jo Jo the Idiot Circus Boy with a pretty new pet his possible sale speech or the Glue sniffing Guarantee fairy brake pad sale The sappy moments in the film contrast sharply with Farley and Spades shenanigans Even after many viewings its still fun to see Farley pour everything he had into the role Richard whats HAPPENING to me\n']

Lemmatize the dataset.

In [285]:
def lemmatize(dataset):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in lines.split()]) for lines in dataset]

trainNeg = lemmatize(trainNeg)
trainPos = lemmatize(trainPos)

testNeg = lemmatize(testNeg)
testPos = lemmatize(testPos)

devPos = lemmatize(devPos)
devNeg = lemmatize(devNeg)

trainPos[:1]

['For fan of Chris Farley this is probably his best film David Spade play the perfect cynical sarcastic yin to Farleys Baby Huey yang Farley achieves stroke of comic genius in his monologue like the Lets say youre driving along the road with your family bit the Jo Jo the Idiot Circus Boy with a pretty new pet his possible sale speech or the Glue sniffing Guarantee fairy brake pad sale The sappy moment in the film contrast sharply with Farley and Spades shenanigan Even after many viewing it still fun to see Farley pour everything he had into the role Richard whats HAPPENING to me']

Create the TF-IDF Vectorizer to pick the features.

In [0]:
vectorizer = TfidfVectorizer(binary=True, max_features=5000, ngram_range=(1, 3), stop_words=stop.words('english'))

Combine the negative and positive reviews.

Create the `trainY` and `testY` lists containing the list of known reviews.

In [0]:
train = trainNeg + trainPos
test = testNeg + testPos

trainY = [1 if i < 7517 else 0 for i in range(15000)]
testY = [1 if i < 2501 else 0 for i in range(5000)]

Train the vectorizer.

Transform the databases.

In [288]:
vectorizer.fit(train)
print(len(vectorizer.get_feature_names()))

train = vectorizer.transform(train)
test = vectorizer.transform(test)

train.shape

5000


(15000, 5000)

Create a Random Forest Classifier and use it for the model.

Train the classifier with the dataset.

In [0]:
classifier = RandomForestClassifier()
classifier.fit(train, trainY)

prediction = classifier.predict(test)

Print the accuracy.

In [290]:
print('Accuracy = ',round(accuracy_score(testY, prediction),2))
print('F1 score = ', round(f1_score(testY, prediction),2))
print('Recall = ', round(recall_score(testY, prediction),2))
print('Precision = ', round(precision_score(testY, prediction),2))

Accuracy =  0.84
F1 score =  0.84
Recall =  0.83
Precision =  0.84
